In [ ]:
import pandas as pd
import glob
import inc.functions as fn
from inc.indicators import apply_all_indicators

# def load_crypto_data(symbol="BTC-USD", folder_path="data/crypto"):
#     """
#     Loads and concatenates CSV files for a given crypto symbol.
    
#     Parameters:
#         symbol (str): The symbol to load (default "BTC-USD").
#         folder_path (str): Path to the folder containing CSV files.
    
#     Returns:
#         pd.DataFrame: Combined DataFrame sorted by datetime.
#     """
#     pattern = f"{folder_path}/{symbol.replace('-', '_')}_*.csv"
#     files = glob.glob(pattern)

#     if not files:
#         raise ValueError(f"No files found for {symbol} in {folder_path}")

#     dfs = [pd.read_csv(file) for file in files]
#     df = pd.concat(dfs, ignore_index=True)

#     df['Datetime'] = pd.to_datetime(df['Datetime'])
#     df = df.sort_values(['Datetime']).reset_index(drop=True)  # <- Keep all rows, just sort

#     return df

def generate_signal(row, rsi_buy_threshold=40, rsi_sell_threshold=65):
    if row['Crossover'] == 'Bullish' and row['RSI'] < rsi_buy_threshold:
        return 'BUY'
    elif row['Crossover'] == 'Bearish' and row['RSI'] > rsi_sell_threshold:
        return 'SELL'
    else:
        return 'HOLD'


def backtest_signals(df, initial_balance=1000):
    """
    Simulates trading based on generated signals.

    Parameters:
        df (pd.DataFrame): DataFrame with 'Signal' and 'Close' columns.
        initial_balance (float): Starting cash balance.

    Returns:
        float: Final account balance after simulation.
        list: History of account values for plotting if needed.
    """
    balance = initial_balance
    position = 0  # Number of coins held
    account_history = []

    for i, row in df.iterrows():
        price = row['Close']
        signal = row['Signal']

        if signal == 'BUY' and balance > 0:
            position = balance / price  # Buy as much as possible
            balance = 0
        elif signal == 'SELL' and position > 0:
            balance = position * price  # Sell everything
            position = 0

        # Track account value at each step
        account_value = balance + (position * price)
        account_history.append(account_value)

    # Final value (if still holding position, it will be liquidated at last price)
    final_value = balance + (position * df['Close'].iloc[-1])

    return final_value, account_history

def auto_backtest(symbols, folder_path="data/crypto", initial_balance=1000):
    """
    Automatically backtests multiple crypto symbols.

    Parameters:
        symbols (list): List of symbol strings.
        folder_path (str): Path to the folder with CSV files.
        initial_balance (float): Starting cash for each symbol.

    Returns:
        pd.DataFrame: Summary of final balances for each symbol.
    """
    results = []

    for symbol in symbols:
        try:
            df = load_crypto_data(symbol=symbol, folder_path=folder_path)
            df = apply_all_indicators(df)
            df['Signal'] = df.apply(generate_signal, axis=1)
            final_balance, history = backtest_signals(df, initial_balance=initial_balance)

            results.append({
                'Symbol': symbol,
                'Final_Balance': final_balance,
                'Profit': final_balance - initial_balance,
                'Profit_%': ((final_balance - initial_balance) / initial_balance) * 100
            })
        
        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    summary_df = pd.DataFrame(results)
    return summary_df

def live_audit(symbols, folder_path="data/crypto"):
    """
    Provides the latest signal for each symbol for real-time auditing.

    Parameters:
        symbols (list): List of symbols to audit.
        folder_path (str): Where CSV files are stored.

    Returns:
        pd.DataFrame: Latest signal snapshot for each symbol.
    """
    audit_results = []

    for symbol in symbols:
        try:
            df = load_crypto_data(symbol=symbol, folder_path=folder_path)
            df = apply_all_indicators(df)
            df['Signal'] = df.apply(generate_signal, axis=1)

            latest = df.iloc[-1]  # Most recent row
            audit_results.append({
                'Symbol': symbol,
                'Datetime': latest['Datetime'],
                'Close': latest['Close'],
                'Signal': latest['Signal'],
                'RSI': latest['RSI'],
                'MACD': latest['MACD'],
                'Signal_Line': latest['Signal_Line'],
                'MACD_Diff': latest['MACD_Diff']
            })

        except Exception as e:
            print(f"Error auditing {symbol}: {e}")

    audit_df = pd.DataFrame(audit_results)
    return audit_df

# ===
def optimize_rsi_thresholds(df, buy_range=(30, 50, 5), sell_range=(60, 80, 5), initial_balance=1000):
    """
    Finds the best RSI buy/sell thresholds by backtesting different combinations.

    Parameters:
        df (pd.DataFrame): DataFrame with indicators already applied.
        buy_range (tuple): (start, end, step) for buy RSI threshold testing.
        sell_range (tuple): (start, end, step) for sell RSI threshold testing.
        initial_balance (float): Starting balance for backtest.

    Returns:
        dict: Best parameters and corresponding performance.
    """
    best_result = {
        'buy_threshold': None,
        'sell_threshold': None,
        'final_balance': -float('inf')
    }

    for buy_threshold in range(*buy_range):
        for sell_threshold in range(*sell_range):
            temp_df = df.copy()
            temp_df['Signal'] = temp_df.apply(
                lambda row: generate_signal(row, rsi_buy_threshold=buy_threshold, rsi_sell_threshold=sell_threshold),
                axis=1
            )
            final_balance, _ = backtest_signals(temp_df, initial_balance=initial_balance)

            if final_balance > best_result['final_balance']:
                best_result = {
                    'buy_threshold': buy_threshold,
                    'sell_threshold': sell_threshold,
                    'final_balance': final_balance
                }

    return best_result

symbols = [
    "BTC-USD", "ETH-USD", "SOL-USD", "LTC-USD",
    "ADA-USD", "AVAX-USD", "DOGE-USD", "MATIC-USD",
    "XRP-USD", "PEPE-USD","XLM-USD"
]
# for symbol in symbols:
#     df = fn.load_crypto_data(symbol)
#     df = apply_all_indicators(df)
#     df = create_profit_labels(df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib  # For saving models

for symbol in symbols:
    # 1. Load and prepare
    print(f"Loading data for {symbol}...")
    df = fn.load_crypto_data(symbol)
    df = apply_all_indicators(df)
    df = create_profit_labels(df)

    # 2. Select Features and Target
    df['Crossover_Encoded'] = df['Crossover'].map({'Bullish': 1, 'Bearish': -1}).fillna(0)
    #feature_cols = ['RSI', 'MACD', 'ATR', 'VWAP_1m', 'Volume_SMA', 'Spread']  # You can expand this
    feature_cols = [
        'RSI', 'MACD', 'MACD_Diff', 'MACD_ROC', 'ATR',
        '%K', '%D', 'OBV', 'VWAP_15m',
        'EMA_5', 'EMA_13', 'EMA_26',
        'Volume_SMA', 'Spread',
        'Fib_61.8%', 'Fib_100.0%', 'Fib_161.8%','Crossover_Encoded'
    ]
    
    X = df[feature_cols]
    y = df['Label']

    label_counts = y.value_counts()

    if (label_counts < 2).any():
        print(f"Skipping {symbol} due to insufficient label distribution:\n{label_counts}")
        continue  # Skip model training for this symbol

    # 3. Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 4. Split train/test
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

    # 5. Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 6. Save model and scaler (optional but recommended)
    joblib.dump(model, f"inc/models/{symbol.replace('-', '_')}_rf_model.pkl")
    joblib.dump(scaler, f"inc/models/{symbol.replace('-', '_')}_scaler.pkl")

    print(f"Finished training model for {symbol}")


Loading data for BTC-USD...
Finished training model for BTC-USD
Loading data for ETH-USD...
Finished training model for ETH-USD
Loading data for SOL-USD...
Finished training model for SOL-USD
Loading data for LTC-USD...
Finished training model for LTC-USD
Loading data for ADA-USD...
Finished training model for ADA-USD
Loading data for AVAX-USD...
Finished training model for AVAX-USD
Loading data for DOGE-USD...
Finished training model for DOGE-USD
Loading data for MATIC-USD...
Finished training model for MATIC-USD
Loading data for XRP-USD...
Finished training model for XRP-USD
Loading data for PEPE-USD...
Finished training model for PEPE-USD
Loading data for XLM-USD...
Finished training model for XLM-USD


In [ ]:
df['Symbol'].unique()

In [ ]:
df= fn.load_crypto_data(symbol="BTC-USD", folder_path="data/crypto")

# After loading your data
df = apply_all_indicators(df)
df['Signal'] = df.apply(generate_signal, axis=1)

df.tail(3)

In [ ]:
final_balance, history = backtest_signals(df)
print(f"Final Balance: ${final_balance:.2f}")


In [ ]:
symbols = [
    "BTC-USD", "ETH-USD", "SOL-USD", "LTC-USD",
    "ADA-USD", "AVAX-USD", "DOGE-USD", "MATIC-USD",
    "XRP-USD", "PEPE-USD","XLM-USD"
]

summary = auto_backtest(symbols)
print(summary)


In [ ]:
audit = live_audit(symbols)
print(audit)


In [ ]:
df = fn.load_crypto_data(symbol="BTC-USD")      # or whatever symbol
df = apply_all_indicators(df)

best_rsi_settings = optimize_rsi_thresholds(df)
print(best_rsi_settings)


In [ ]:
for symbol in symbols:
    df = fn.load_crypto_data(symbol)
    df = apply_all_indicators(df)
    best_settings = optimize_rsi_thresholds(df)
    print(f"{symbol}: {best_settings}")


In [3]:
# After your create_profit_labels step
# df = create_profit_labels(df, profit_threshold=0.005, loss_threshold=-0.005, future_window=5)
for symbol in symbols:
    df = fn.load_crypto_data(symbol)
    df = fn.apply_all_indicators(df)
    df = fn.create_profit_labels(df)
    summary = fn.backtest_labels_per_symbol(df, 1000, .0035)
    print(summary)

    Symbol  Final_Balance    Profit  Profit_%  Trades
0  BTC-USD     1031.25441  31.25441  3.125441       3
    Symbol  Final_Balance      Profit   Profit_%  Trades
0  ETH-USD    1109.874688  109.874688  10.987469      26
    Symbol  Final_Balance      Profit  Profit_%  Trades
0  SOL-USD    1149.911998  149.911998   14.9912      43
    Symbol  Final_Balance      Profit   Profit_%  Trades
0  LTC-USD    1214.245163  214.245163  21.424516      53
    Symbol  Final_Balance      Profit   Profit_%  Trades
0  ADA-USD    1403.478866  403.478866  40.347887      79
     Symbol  Final_Balance      Profit   Profit_%  Trades
0  AVAX-USD    1272.266047  272.266047  27.226605      87
     Symbol  Final_Balance      Profit   Profit_%  Trades
0  DOGE-USD    1216.909106  216.909106  21.690911      55
      Symbol  Final_Balance      Profit   Profit_%  Trades
0  MATIC-USD    1659.300429  659.300429  65.930043     157
    Symbol  Final_Balance      Profit   Profit_%  Trades
0  XRP-USD    1167.163507  167.

In [9]:
df = fn.load_crypto_data("PEPE-USD")
df = fn.apply_all_indicators(df)
df = fn.create_profit_labels(df)
# df.to_excel("data/paper_trader/PEPE-USD.xlsx", index=False)
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_convert('America/New_York').dt.tz_localize(None)
df.to_excel("data/paper_trader/PEPE-USD.xlsx", index=False)